<a href="https://colab.research.google.com/github/kmadden9/realTimeML/blob/main/HW/HW2/HW2_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Kevin Madden 
#Due Date: 2/28/2022
#Professor: Dr. Hamed Tabkhi, PH.D.

import torch
from PIL import Image
from torchvision import transforms
import requests
from io import BytesIO
import numpy as np
import torch.optim as optim

#checking if I have GPU connected
torch.cuda.is_available()

True

In [2]:
#3.A
import csv
housing_path = "https://raw.githubusercontent.com/kmadden9/realTimeML/main/Housing_fixed.csv"
house_numpy = np.loadtxt(housing_path, dtype=np.float32, delimiter=",", skiprows=1)
house_numpy

array([[1.330e+07, 7.420e+03, 4.000e+00, 2.000e+00, 3.000e+00, 2.000e+00],
       [1.225e+07, 8.960e+03, 4.000e+00, 4.000e+00, 4.000e+00, 3.000e+00],
       [1.225e+07, 9.960e+03, 3.000e+00, 2.000e+00, 2.000e+00, 2.000e+00],
       ...,
       [1.750e+06, 3.620e+03, 2.000e+00, 1.000e+00, 1.000e+00, 0.000e+00],
       [1.750e+06, 2.910e+03, 3.000e+00, 1.000e+00, 1.000e+00, 0.000e+00],
       [1.750e+06, 3.850e+03, 3.000e+00, 1.000e+00, 2.000e+00, 0.000e+00]],
      dtype=float32)

In [3]:
col_list = next(csv.reader(open('/content/raw.githubusercontent.com/kmadden9/realTimeML/main/Housing_fixed.csv'), delimiter=','))
house_numpy.shape, col_list

((545, 6), ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking'])

In [4]:
house = torch.from_numpy(house_numpy)
house.shape, house.dtype

(torch.Size([545, 6]), torch.float32)

In [5]:
data = house[:, 1:]
target = house[:, 0]
data_mean = torch.mean(data, dim=0)
data_var = torch.var(data, dim=0)
target_mean = torch.mean(target, dim=0)
target_var = torch.var(target, dim=0)
data_normalized = (data - data_mean) / torch.sqrt(data_var)
target_normalized = (target - target_mean) / torch.sqrt(target_var)

area = torch.stack(tuple(data_normalized[:,0]))
bedrooms = torch.stack(tuple(data_normalized[:,1]))
bathrooms = torch.stack(tuple(data_normalized[:,2]))
stories = torch.stack(tuple(data_normalized[:,3]))
parking = torch.stack(tuple(data_normalized[:,4]))

In [6]:
target

tensor([13300000., 12250000., 12250000., 12215000., 11410000., 10850000.,
        10150000., 10150000.,  9870000.,  9800000.,  9800000.,  9681000.,
         9310000.,  9240000.,  9240000.,  9100000.,  9100000.,  8960000.,
         8890000.,  8855000.,  8750000.,  8680000.,  8645000.,  8645000.,
         8575000.,  8540000.,  8463000.,  8400000.,  8400000.,  8400000.,
         8400000.,  8400000.,  8295000.,  8190000.,  8120000.,  8080940.,
         8043000.,  7980000.,  7962500.,  7910000.,  7875000.,  7840000.,
         7700000.,  7700000.,  7560000.,  7560000.,  7525000.,  7490000.,
         7455000.,  7420000.,  7420000.,  7420000.,  7350000.,  7350000.,
         7350000.,  7350000.,  7343000.,  7245000.,  7210000.,  7210000.,
         7140000.,  7070000.,  7070000.,  7035000.,  7000000.,  6930000.,
         6930000.,  6895000.,  6860000.,  6790000.,  6790000.,  6755000.,
         6720000.,  6685000.,  6650000.,  6650000.,  6650000.,  6650000.,
         6650000.,  6650000.,  6629000

In [7]:
n_samples = target.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
train_indices, val_indices

(tensor([252, 341, 177,   0,  47, 534,  42, 240, 543, 111, 383,  64, 136, 303,
         195, 367, 293, 473,   1,  27, 377, 538, 261, 126, 436,  54, 110, 127,
         374, 544, 258, 410, 125, 226, 267, 145,   3, 257,  92, 196, 477, 420,
         418, 109, 133, 247, 385,  77, 192, 130, 312, 238, 260, 149, 443,  52,
         516, 457, 248, 510, 500, 508, 347,  83,  67, 212, 102, 202, 284, 467,
         480,  37,  17, 280,  13, 206,  48, 527, 483, 182, 499, 314,  58, 160,
         237,  78, 173, 450, 461, 236, 287, 318, 201, 432, 504, 517, 540, 442,
          99, 532, 362, 403, 423, 357, 186, 379, 366, 404, 189, 138, 498, 274,
         386, 276, 279, 235,  44, 108,  60, 115, 264,  40,  80, 140, 511,  16,
         488, 424,  94, 334, 211, 225, 430, 224, 233, 271, 179, 123, 425, 162,
         281, 205, 208, 460, 291, 354, 301, 396, 302, 166, 382, 327, 117, 199,
         335,  79, 384, 105, 227, 419, 531, 494, 241, 400, 157, 143, 402, 167,
         373, 537, 255, 155, 132, 496, 118,  19, 539

In [8]:
train_area = area[train_indices]
train_bedrooms = bedrooms[train_indices]
train_bathrooms = bathrooms[train_indices]
train_stories = stories[train_indices]
train_parking = parking[train_indices]

train_data = data_normalized[train_indices]
train_price = target_normalized[train_indices]

eval_area = area[val_indices]
eval_bedrooms = bedrooms[val_indices]
eval_bathrooms = bathrooms[val_indices]
eval_stories = stories[val_indices]
eval_parking = parking[val_indices]

eval_data = data_normalized[val_indices]
eval_price = target_normalized[val_indices]




In [9]:
def model(area, bedrooms, bathrooms, stories, parking, w1, w2, w3, w4, w5, b):
  return w5 * area + w4 * bedrooms + w3 * bathrooms + w2 * stories + w1 * parking + b

In [10]:
def loss_fn(price_pred, target_normalized):
  squared_diffs = (price_pred - target_normalized)**2
  return squared_diffs.mean()

In [11]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-5
optimizer = optim.SGD([params], lr=learning_rate)

In [12]:
price_pred = model(area, bedrooms, bathrooms, stories, parking, *params)
loss = loss_fn(price_pred, target_normalized)

loss.backward()
optimizer.step()

params

tensor([9.9997e-01, 9.9997e-01, 9.9997e-01, 9.9997e-01, 9.9998e-01, 3.0920e-12],
       requires_grad=True)

In [13]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

price_train = model(train_area, train_bedrooms, train_bathrooms, train_stories, train_parking, *params)
train_loss = loss_fn(price_train, train_price)

price_eval = model(eval_area, eval_bedrooms, eval_bathrooms, eval_stories, eval_parking, *params)
eval_loss = loss_fn(price_eval, eval_price)

optimizer.zero_grad()
train_loss.backward()
optimizer.step()

params

tensor([0.9738, 0.9702, 0.9700, 0.9660, 0.9762, 0.0015], requires_grad=True)

In [14]:
def training_loop(n_epochs, optimizer, params, train_area, train_bedrooms, train_bathrooms, train_stories, train_parking, train_price,
                  eval_area, eval_bedrooms, eval_bathrooms, eval_stories, eval_parking, eval_price):
  for epoch in range(1, n_epochs + 1):
    price_train = model(train_area, train_bedrooms, train_bathrooms, train_stories, train_parking, *params)
    train_loss = loss_fn(price_train, train_price)

    price_eval = model(eval_area, eval_bedrooms, eval_bathrooms, eval_stories, eval_parking, *params)
    eval_loss = loss_fn(price_eval, eval_price)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
        f" Validation loss {eval_loss.item():.4f}")
  
  return params

In [15]:
#2.b
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = .001
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
  n_epochs = 5000,
  optimizer = optimizer,
  params = params,
  train_area = train_area,
  train_bedrooms = train_bedrooms,
  train_bathrooms = train_bathrooms,
  train_stories = train_stories,
  train_parking = train_parking,
  train_price = train_price,
  eval_area = eval_area,
  eval_bedrooms = eval_bedrooms,
  eval_bathrooms = eval_bathrooms,
  eval_stories = eval_stories,
  eval_parking = eval_parking,
  eval_price = eval_price)

Epoch 1, Training loss 5.9623, Validation loss 6.3801
Epoch 2, Training loss 5.9204, Validation loss 6.3363
Epoch 3, Training loss 5.8788, Validation loss 6.2928
Epoch 500, Training loss 0.5919, Validation loss 0.5656
Epoch 1000, Training loss 0.4671, Validation loss 0.3699
Epoch 1500, Training loss 0.4624, Validation loss 0.3547
Epoch 2000, Training loss 0.4618, Validation loss 0.3531
Epoch 2500, Training loss 0.4616, Validation loss 0.3531
Epoch 3000, Training loss 0.4616, Validation loss 0.3533
Epoch 3500, Training loss 0.4616, Validation loss 0.3535
Epoch 4000, Training loss 0.4616, Validation loss 0.3536
Epoch 4500, Training loss 0.4616, Validation loss 0.3537
Epoch 5000, Training loss 0.4616, Validation loss 0.3537


tensor([0.1920, 0.2216, 0.3334, 0.0861, 0.3880, 0.0123], requires_grad=True)

In [16]:
import torch.nn as nn

linear_model = nn.Linear(5, 1)
linear_model(data_normalized)

tensor([[ 8.3132e-01],
        [ 2.8802e+00],
        [ 8.3531e-01],
        [ 9.4112e-01],
        [-2.1493e-01],
        [ 1.5236e+00],
        [ 1.7557e+00],
        [-1.0937e+00],
        [-2.8639e-01],
        [ 1.4678e+00],
        [-2.4321e-01],
        [ 1.4103e+00],
        [ 1.8808e-01],
        [ 9.3506e-01],
        [ 2.0944e-01],
        [-6.5692e-02],
        [ 1.8282e-01],
        [ 1.6052e+00],
        [ 1.3986e+00],
        [ 7.8091e-01],
        [ 6.9004e-01],
        [ 8.2187e-01],
        [-4.3664e-01],
        [ 9.7639e-01],
        [ 9.5722e-01],
        [ 6.1557e-01],
        [ 1.0151e+00],
        [-5.2334e-01],
        [-1.1179e-01],
        [ 2.9843e-01],
        [ 1.7129e+00],
        [ 1.0249e+00],
        [ 3.6359e-01],
        [ 1.5673e+00],
        [-1.1596e+00],
        [ 1.7629e+00],
        [ 9.7753e-01],
        [ 9.7350e-01],
        [ 1.1299e+00],
        [ 8.6234e-01],
        [-3.9720e-01],
        [ 9.7724e-01],
        [ 1.8175e+00],
        [ 1

In [17]:
linear_model.weight

Parameter containing:
tensor([[-0.2281, -0.4275,  0.3708,  0.2674,  0.3674]], requires_grad=True)

In [18]:
linear_model.bias

Parameter containing:
tensor([0.2171], requires_grad=True)

In [19]:
x = torch.ones(5)
linear_model(x)

tensor([0.5672], grad_fn=<AddBackward0>)

In [20]:
x = torch.ones(10, 5)
linear_model(x)

tensor([[0.5672],
        [0.5672],
        [0.5672],
        [0.5672],
        [0.5672],
        [0.5672],
        [0.5672],
        [0.5672],
        [0.5672],
        [0.5672]], grad_fn=<AddmmBackward0>)

In [21]:
data_normalized = torch.tensor(data_normalized).unsqueeze(1)
target_normalized = torch.tensor(target_normalized).unsqueeze(1)


data_normalized.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([545, 1, 5])

In [22]:
target_normalized.shape

train_price = torch.tensor(train_price).unsqueeze(1)

eval_price = torch.tensor(eval_price).unsqueeze(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [39]:
target_normalized.shape

torch.Size([545, 1])

In [38]:
eval_price.shape

torch.Size([109, 1])

In [23]:
linear_model = nn.Linear(5, 1)
optimizer = optim.SGD(
  linear_model.parameters(),
  lr=1e-2)

In [24]:
linear_model.parameters()

<generator object Module.parameters at 0x7f21d8c99e50>

In [25]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[ 0.3330, -0.2801,  0.3444,  0.2206,  0.0361]], requires_grad=True),
 Parameter containing:
 tensor([0.3689], requires_grad=True)]

In [26]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_data, train_price,
                  eval_data, eval_price):
  for epoch in range(1, n_epochs + 1):
    price_train = model(train_data)
    train_loss = loss_fn(price_train, train_price)

    price_eval = model(eval_data)
    eval_loss = loss_fn(price_eval, eval_price)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch == 1 or epoch % 50 == 0:
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
            f" Validation loss {eval_loss.item():.4f}")


In [59]:
linear_model = nn.Linear(5, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
  n_epochs = 3000,
  optimizer = optimizer,
  model = linear_model,
  loss_fn = nn.MSELoss(),
  train_data = train_data,
  train_price = train_price,
  eval_data = eval_data,
  eval_price = eval_price)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 1.8364, Validation loss 1.3323
Epoch 50, Training loss 0.5587, Validation loss 0.3374
Epoch 100, Training loss 0.4729, Validation loss 0.3298
Epoch 150, Training loss 0.4635, Validation loss 0.3426
Epoch 200, Training loss 0.4620, Validation loss 0.3486
Epoch 250, Training loss 0.4617, Validation loss 0.3512
Epoch 300, Training loss 0.4616, Validation loss 0.3525
Epoch 350, Training loss 0.4616, Validation loss 0.3531
Epoch 400, Training loss 0.4616, Validation loss 0.3534
Epoch 450, Training loss 0.4616, Validation loss 0.3535
Epoch 500, Training loss 0.4616, Validation loss 0.3536
Epoch 550, Training loss 0.4616, Validation loss 0.3537
Epoch 600, Training loss 0.4616, Validation loss 0.3537
Epoch 650, Training loss 0.4616, Validation loss 0.3537
Epoch 700, Training loss 0.4616, Validation loss 0.3537
Epoch 750, Training loss 0.4616, Validation loss 0.3537
Epoch 800, Training loss 0.4616, Validation loss 0.3537
Epoch 850, Training loss 0.4616, Validation loss 0.

In [60]:
seq_model = nn.Sequential(
            nn.Linear(1, 13),
            nn.Tanh(),
            nn.Linear(13, 1))

seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [61]:
[param.shape for param in seq_model.parameters()]

[torch.Size([13, 1]), torch.Size([13]), torch.Size([1, 13]), torch.Size([1])]

In [62]:
for name, param in seq_model.named_parameters():
  print(name, param.shape)

0.weight torch.Size([13, 1])
0.bias torch.Size([13])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [63]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
  ('hidden_linear', nn.Linear(5, 8)),
  ('hidden_activation', nn.Tanh()),
  ('hidden_linear2', nn.Linear(8, 8)),
  ('hidden_activation2', nn.Tanh()),
  ('output_linear', nn.Linear(8, 1)),
]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=5, out_features=8, bias=True)
  (hidden_activation): Tanh()
  (hidden_linear2): Linear(in_features=8, out_features=8, bias=True)
  (hidden_activation2): Tanh()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [64]:
for name, param in seq_model.named_parameters():
  print(name, param.shape)  

hidden_linear.weight torch.Size([8, 5])
hidden_linear.bias torch.Size([8])
hidden_linear2.weight torch.Size([8, 8])
hidden_linear2.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [65]:
seq_model.output_linear.bias

Parameter containing:
tensor([0.3264], requires_grad=True)

In [66]:
##train_price_fixed = train_price.clone().detach()
#train_price_fixed.shape
train_price = torch.tensor(train_price).squeeze(1)
train_price = torch.tensor(train_price).unsqueeze(1)
train_price.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


torch.Size([436, 1])

In [67]:
eval_price = torch.tensor(eval_price).squeeze(1)
eval_price = torch.tensor(eval_price).unsqueeze(1)
eval_data.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([109, 5])

In [68]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)

training_loop(
  n_epochs = 200,
  optimizer = optimizer,
  model = seq_model,
  loss_fn = nn.MSELoss(),
  train_data = train_data,
  train_price = train_price,
  eval_data = eval_data,
  eval_price = eval_price)

  #return F.mse_loss(input, target, reduction=self.reduction)
print('output', seq_model(eval_data))
print('answer', eval_price)
print('hidden', seq_model.hidden_linear.weight.grad)

Epoch 1, Training loss 0.9533, Validation loss 0.6652
Epoch 50, Training loss 0.6395, Validation loss 0.4095
Epoch 100, Training loss 0.5235, Validation loss 0.3386
Epoch 150, Training loss 0.4982, Validation loss 0.3387
Epoch 200, Training loss 0.4918, Validation loss 0.3415
output tensor([[ 1.2026],
        [ 1.1263],
        [-0.3203],
        [-0.8566],
        [ 0.1454],
        [ 1.2781],
        [ 0.2238],
        [ 0.3888],
        [ 1.2840],
        [ 0.5316],
        [-0.9472],
        [ 1.4317],
        [ 0.8168],
        [ 0.1698],
        [-0.2805],
        [-0.9684],
        [ 0.9769],
        [ 0.5250],
        [-0.6936],
        [-0.2401],
        [ 0.6033],
        [ 0.7582],
        [-0.5531],
        [-1.0901],
        [ 0.2595],
        [ 1.2853],
        [-0.2987],
        [-1.0132],
        [ 0.8806],
        [ 1.3809],
        [-0.5386],
        [-0.0577],
        [ 0.6547],
        [ 1.2366],
        [ 0.0747],
        [-0.7315],
        [-1.0623],
        [ 1.3